# 02-01 Dimensionen Demo

## Hinweise zur Übung

Ziel dieser Übung ist die beispielhafte Erzeugung von Dimensionstabellen für das Data Warehouse-Szenario. Hierzu nutzen wir die Inhalte der Staging-Datenbanken als Datenquelle und transformieren daraus die Dimensionsdaten und liefern sie in die Reporting-Datenbank aus.

Die Übung setzt auf den Ergebnissen der Extraktions (Übung 01) auf. Da in der Übung nur einzelne Tabellen in die Staging-Datenbanken übertragen wurden, setzen wir hier mit einem einheitlichen Stand aus vorbereiteten SQLite-Datenbanken auf, in die alle TSV-Dateien bereits importiert wurden.

Die Tabellenstruktur der zu erstellenden Dimensionstabellen inkl. der Spalten (nicht jedoch Datentypen) geht aus dem ER-Diagramm im `data`-Verzeichnis hervor. Die Datentypen wiederum können in den Dokumentationen der jeweiligen Qeulldatenbereiche (Bielefeld, Mannheim, Dimensionen) nachgeschlagen werden.

## Nutzung des ELT- statt ETL-Ansatzes

In dieser Übung laden wir Daten nicht mehr aus CSV-Dateien, sondern zwischen Schemata (hier in SQLite: Teil-Datenbanken) der gleichen übergeordneten Datenbank hin und her.

Hierzu können wir die SQL Funktionen `CREATE TABLE ... AS SELECT ...` bzw. `INSERT INTO ... SELECT ...` nutzen, um Daten direkt von einem Teil der Datenbank zu lesen und in eine andere Tabelle der gleichen Datenbank (hier in einem anderen Schema) zu schreiben.

Da hierbei kein ETL-Werkzeug benötigt wird, das die Teilschritte des extrahierens, transformierens und ladens getrennt voneinander ausführt, wird diese Vorgehensweise auch als **ELT** (Extract, Load **and** Transform) bezeichnet. Wir setzen entsprechend in diesem Notebook nicht PETL, sondern nur SQL ein.

## Konfiguration des Notebooks

In [ ]:
# Ggf. fehlende Pakete installieren
!pip install --quiet ipython-sql

In [ ]:
import os
import sys
import urllib.request
%load_ext sql

In [ ]:
# Konfiguration
base_url_quellen = "https://raw.githubusercontent.com/fau-lmi/lct-ehealth/main/07-ETL+DWH/data"
base_url_staging = "./"

In [ ]:
# SQlite-Datenbanken aus Github auf den Jupyter-Server herunterladen
urllib.request.urlretrieve(base_url_quellen + "/datenbanken/stg_bielefeld.sqlite",  "stg_bielefeld.sqlite")
urllib.request.urlretrieve(base_url_quellen + "/datenbanken/stg_mannheim.sqlite",   "stg_mannheim.sqlite")
urllib.request.urlretrieve(base_url_quellen + "/datenbanken/stg_dimensions.sqlite", "stg_dimensions.sqlite")
urllib.request.urlretrieve(base_url_quellen + "/datenbanken/staging.sqlite",        "staging.sqlite")


In [ ]:
# Datenbankverbindung als Pfad (für das ETL) & iPython SQL (für die Abfragen) herstellen
db_path_stg_bielefeld  = base_url_staging + "stg_bielefeld.sqlite"
db_path_stg_mannheim   = base_url_staging + "stg_mannheim.sqlite"
db_path_stg_dimensions = base_url_staging + "stg_dimensions.sqlite"
db_path_staging        = base_url_staging + "staging.sqlite"
db_path_reporting      = base_url_staging + "reporting.sqlite"

db_url_stg_bielefeld  = "sqlite:///" + db_path_stg_bielefeld
db_url_stg_mannheim   = "sqlite:///" + db_path_stg_mannheim
db_url_stg_dimensions = "sqlite:///" + db_path_stg_dimensions
db_url_staging        = "sqlite:///" + db_path_staging
db_url_reporting      = "sqlite:///" + db_path_reporting

%sql $db_url_stg_bielefeld
%sql $db_url_stg_mannheim
%sql $db_url_stg_dimensions
%sql $db_url_staging
%sql $db_url_reporting

In [ ]:
%%sql $db_url_staging
ATTACH DATABASE :db_path_stg_bielefeld  AS bielefeld;
ATTACH DATABASE :db_path_stg_mannheim   AS mannheim;
ATTACH DATABASE :db_path_stg_dimensions AS dimensions;
ATTACH DATABASE :db_path_reporting      AS reporting;

## Dimensionstabelle D_AUFNAHMEGRUND

Der Aufnahmegrund ist eine wichtige Information für die Auswertung klinischer Versorgungsdaten ("Warum wurde ein Patient aufgenommen?"). Im deutschen Gesundheitswesen findet man hierzu hauptsächlich abrechnungsrelevante Daten, während der in der Übung verwendete Synthea-Datensatz eher medizinische Gründe listet.

Im folgenden Block erzeugen wir die Dimensionstabelle D_AUFNAHMEGRUND. Ihre Quelldaten können wir direkt aus den Rohdatentabellen FAELLE/ENCOUNTER der beiden Quelldatenbanken entnehmen. Sie enthalten SNOMED-Codes und -Bezeichner für die Aufnahmegründe. Da in beiden Quelldatenbanken der gleiche Katalog (SNOMED CT) für die Aufnahmegründe verwendet wird, können wir eine gemeinsame Dimension mit den gleichen Identifiern anlegen.

Da diese Dimension nur eine Ebene mit wenigen Einträgen hat, gibt es keine Hierarchie, und sowohl die Benennung der Spalten als auch die Abfragen zur Generierung sind sehr überschaubar.

### 1. Zieltabelle anlegen

Hierzu legen wir zunächst mit einem CREATE-Table die Tabelle in der Reporting-Datenbank (leer) an.

In [ ]:
%%sql $db_url_staging
DROP TABLE IF EXISTS reporting.d_aufnahmegrund;

CREATE TABLE reporting.d_aufnahmegrund (
  aufnahmegrund_id   VARCHAR(20),
  aufnahmegrund_name VARCHAR(100)
);

### 2. Daten aus den Quelltabellen auslesen & in Zieltabelle schreiben

In diesem Schritt lesen wir mit `SELECT`-Statements die Daten aus den Quelltabellen beider Standorte aus.

Wir benennen bei der Abfrage die Spalten mit der `AS`-Klausel nach den Vorgeben unseres Datenmodells um.

Um eindeutige Einträge in der Zieltabelle zu erhalten, nutzen wir die `DISTINCT`-Klausel.

Da wir Daten aus zwei Quelltabellen zusammenführen, müssen wir zwei getrennte SQL-Statements nutzen, die wir mit einem `UNION`-Statement verbinden. Die abgefragten Spalten beider Statements müssen dabei identische Namen und Datentypen haben (ggf. müssen hierzu mit `AS` Aliasse definiert werden). `UNION` stellt automatisch sicher, dass Zeilen, die in beiden Teilabfragen identisch sind, zusammengeführt werden (im Sinne eines `DISTINCT` über beide Abfragen zusammen). Wenn Sie stattdessen `UNION ALL` verwenden, bleiben doppelte Einträge erhalten, was aber in einer Dimensionstabelle die Bedingung, dass alle Einträge disjunkt sein müssen, verletzen würde.

In [ ]:
%%sql $db_url_staging
DELETE FROM reporting.d_aufnahmegrund;

INSERT INTO reporting.d_aufnahmegrund (aufnahmegrund_id, aufnahmegrund_name)

SELECT DISTINCT
       aufnahmegrund      AS aufnahmegrund_id,
       aufnahmegrund_text AS aufnahmegrund_name
  FROM bielefeld.faelle

UNION

SELECT DISTINCT
       code               AS aufnahmegrund_id,
       description        AS aufnahmegrund_name
  FROM mannheim.encounters;

### 3. Ergebniskontrolle

Anschließend können wir mit einer `SELECT`-Abfrage überprüfen, ob die Tabelle korrekt erstellt wurde.


In [ ]:
%%sql $db_url_staging
SELECT *
  FROM reporting.d_aufnahmegrund
 LIMIT 10

## Dimensionstabelle D_LOINC

Im Folgenden erstellen wir eine etwas komplexere Dimensionstabelle mit zwei Ebenen für die Gliederung der Laborbefunde im Data Warehouse.

LOINC (Logical Observation Identifiers Names and Codes) ist eine internationale Terminologie, die zunächst im Laborbereich, inzwischen aber sehr breit für jegliche "Beobachtungen" (Observations) durchgesetzt hat. LOINC ist auch die Vorgabe für die semantische Codierung von Laborbefunden in der elektronischen Patientenakte im Deutschen Gesundheitswesen. LOINC ist eine Mehrachsige Terminologie, die neben eindeutigen Bezeichnern für die Observations auch eine Reihe strukturierter Merkmale z.B. zum untersuchten Parameter und der Art der Messung macht. Dies trägt zur Vergleichbarkeit von Befunddaten verschiedener Labore & Standorte bei. Weitere Informationen zu LOINC gibt es auf der [LOINC-Homepage](https://loinc.org/get-started/).

Als Rohdaten verwenden wir hier nicht die Quelldaten der beiden Standorte, sondern den im Dimensionsbereich bereitgestellten Auszug der LOINC-Hierarchie, der zusätzliche Attribute enthält.

### 1. Zieltabelle anlegen

Hierzu legen wir zunächst mit einem CREATE-Table die Tabelle in der Reporting-Datenbank (leer) an.

In [ ]:
%%sql $db_url_staging
DROP TABLE IF EXISTS reporting.d_loinc;

CREATE TABLE reporting.d_loinc (
  loinc_klasse_id   VARCHAR(10),
  loinc_klasse_name VARCHAR(10),
  loinc_id          VARCHAR(15),
  loinc_name        VARCHAR(50),
  loinc_langname    VARCHAR(200),
  loinc_component   VARCHAR(50),
  loinc_property    VARCHAR(50),
  loinc_timeaspect  VARCHAR(10),
  loinc_system      VARCHAR(50),
  loinc_scaletype   VARCHAR(10),
  loinc_methodtype  VARCHAR(150)
);

### 2. Daten aus der Quelltabelle auslesen & in Zieltabelle schreiben

Die Dimension D_LOINC hat laut DWH-Datenmodell die beiden folgenden Ebenen:
* LOINC_KLASSE: übergeordnete Klasse eines Parameters
* LOINC: Parameter & dazugehörige Attribute

Die LOINC-Klasse ist in den Quelldaten (wie oben bei der Fallart) nur mit einem textuellen Code, jedoch keiner ID abgebildet. Da der Code sprechend ist, tragen wir den Text der Klasse identisch in die _ID- und _NAME-Spalten ein.

Da die Quelldaten für beide Ebenen der Hierarchie aus der gleichen Quelltabelle stammen, besteht die Aufgabe hier nur darin, die Spalten gemäß den Vorgaben des DWH-Datenmodells umzubenennen und in die Zieltabelle per `INSERT INTO  ... SELECT ...` auszuliefern.

In [ ]:
%%sql $db_url_staging
DELETE FROM reporting.d_loinc;

INSERT INTO reporting.d_loinc (loinc_klasse_id, loinc_klasse_name, loinc_id, loinc_name, loinc_langname, loinc_component, loinc_property, loinc_timeaspect, loinc_system, loinc_scaletype, loinc_methodtype)

SELECT class            AS loinc_klasse_id,
       class            AS loinc_klasse_name,
       loinc_num        AS loinc_id,
       shortname        AS loinc_name,
       long_common_name AS loinc_langname,
       component        AS loinc_component,
       property         AS loinc_property,
       time_aspct       AS loinc_timeaspect,
       system           AS loinc_system,
       scale_typ        AS loinc_scaletype,
       method_typ       AS loinc_methodtype
  FROM dimensions.loinc;

### 3. Ergebniskontrolle

Anschließend können wir mit einer `SELECT`-Abfrage überprüfen, ob die Tabelle korrekt erstellt wurde.

In [ ]:
%%sql $db_url_staging
SELECT *
  FROM reporting.d_loinc
 LIMIT 10